In [1]:
import pandas as pd
from datetime import datetime
from emoji import emojize

### Задание 1
Используем файл keywords.csv.
 
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.
 
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
 
    'Центр': ['москва', 'тула', 'ярославль'],
 
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
 
}
 
Результат классификации запишите в отдельный столбец region.

In [2]:
geo_data = {
            'Центр': ['москва', 'тула', 'ярославль'],

            'Северо-Запад': ['петербург', 'псков', 'мурманск'],

            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
                 
}

#### Решение

для решения данной задачи я написал две функции geo_tags_search_engine и find_key
при решении использовался следующий алгоритм:
- разбиваем столбец keyword на слова и возвращаем результат в новый столбец keyword_split
- функцией find_key проходим по столбцу keyword_split и на каждой итерации внутри функции вызываем функцию 
  geo_tags_search_engine которая возвращает название региона или undef если геотегов нет
  

In [3]:
with_geo_tags = pd.read_csv('keywords.csv') 

In [4]:
#создаем новый столбец и пробрасываем в него разбитые слова из столбца keyword
with_geo_tags['keyword_split'] = with_geo_tags.apply(lambda x: x['keyword'].split(' '), axis=1)

In [5]:
#отлично все работает
with_geo_tags.head()

,keyword,shows,keyword_split
0,вк,64292779,[вк]
1,одноклассники,63810309,[одноклассники]
2,порно,41747114,[порно]
3,ютуб,39995567,[ютуб]
4,вконтакте,21014195,[вконтакте]


In [6]:
def geo_tags_search_engine(city):
    """
    функция geo_tags_search_engine принимает на вход значение
    и проверяет есть ли это значение в словаре geo_data
    если данне сопадают то функция возвращает ключ 
    (в данном примере название  региона) если нет то 'undef'
    """
    
    for region, city_list in geo_data.items():
        if city in city_list:
            return region
    return 'undef'

In [7]:
def find_key(city_in_series):
    """
    функция find_key на вход принимает данные из столбца 
    'keyword_split' и с помощью итератора for проходит 
    по каждому значениею и на каждоей итерации внутри себя 
    вызывает функцию geo_tags_search_engine которая делает магию
    (описание geo_tags_search_engine дано выше)
    """
    
    for city in city_in_series['keyword_split']:
        return geo_tags_search_engine(city)

In [8]:
#создаем новый столбец в который возвращаем результаты работы функции find_key
with_geo_tags['geo_tags'] = with_geo_tags.apply(find_key, axis=1)

In [9]:
#все сработало, теперь у нас есть столбец geo_tags
with_geo_tags.head()

,keyword,shows,keyword_split,geo_tags
0,вк,64292779,[вк],undef
1,одноклассники,63810309,[одноклассники],undef
2,порно,41747114,[порно],undef
3,ютуб,39995567,[ютуб],undef
4,вконтакте,21014195,[вконтакте],undef


In [10]:
#наведем немного красоты
with_geo_tags.drop('keyword_split', axis=1, inplace=True)

In [11]:
with_geo_tags.head()

,keyword,shows,geo_tags
0,вк,64292779,undef
1,одноклассники,63810309,undef
2,порно,41747114,undef
3,ютуб,39995567,undef
4,вконтакте,21014195,undef


In [12]:
emojize(':thumbs_up:')

'👍'

### Задание 2
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг
 
Результат классификации запишите в столбец class

#### Решение

тут все настолько тривиаль что нет смысла описывать ход решения

In [13]:
rating_df = pd.read_csv('ratings.csv')

In [14]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [15]:
def check_the_rating(row):
    """
    функция check_the_rating классифицирует фильмы
    """
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif row['rating'] <= 4:
        return 'средний рейтинг'
    elif row['rating'] > 4:
        return 'высокий рейтинг'

In [16]:
%%time
rating_df['class'] = rating_df.apply(check_the_rating, axis= 1)

CPU times: user 2.29 s, sys: 8.32 ms, total: 2.29 s
Wall time: 2.3 s


In [17]:
rating_df.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [18]:
rating_df['class'].value_counts()

средний рейтинг    63801
высокий рейтинг    22818
низкий рейтинг     13385
Name: class, dtype: int64

In [13]:
print(emojize(':thumbs_up:'))

👍


### Задание 3
Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [19]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [21]:
#групперуем по userId и считаем сколько каждый пользователь поставил оценок фильмам
ratings_count = ratings.groupby('userId').count().reset_index()[['userId', 'movieId']]

In [22]:
ratings_count.head()

,userId,movieId
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [23]:
#из нашей совокупности делаем выборку пользователей который поставили 100 и более оценок
film_fans_ratings_count = ratings_count[ ratings_count['movieId'] >= 100 ].sort_values('movieId', ascending=False)
film_fans_ratings_count.head()

,userId,movieId
546,547,2391
563,564,1868
623,624,1735
14,15,1700
72,73,1610


In [24]:
film_fans_user_ids = film_fans_ratings_count['userId'].tolist()

In [25]:
#fans_data наши данные с самыми активными киноманами
fans_data = ratings[ratings['userId'].isin(film_fans_user_ids)]

In [37]:
#переводим unixtime в более привычный нам формат YYYY-MM-DD для этого используем pd.to_datetime
fans_data['timestamp'] = pd.to_datetime(fans_data['timestamp'],unit='s')

In [27]:
#выполняем агрегирование по min и max по timestamp для каждого пользователя 
min_max_fans_lifetime = fans_data.groupby('userId').agg(['min', 'max'])['timestamp'].reset_index()

In [28]:
#отлично у нас есть данные о самой первой и последней активности киноманов
min_max_fans_lifetime.head()

,userId,min,max
0,4,2000-02-05 19:25:14,2000-02-08 03:57:54
1,5,2006-11-12 23:10:44,2006-11-12 23:45:45
2,8,2006-07-31 23:42:20,2006-08-01 23:22:07
3,15,2001-08-16 04:47:19,2016-07-24 03:25:35
4,17,2005-09-23 09:43:07,2005-09-23 11:57:20


In [29]:
#определяем lifetime и возвращаем значение в новый столбец
min_max_fans_lifetime['lifetime'] = min_max_fans_lifetime['max'] - min_max_fans_lifetime['min']

In [33]:
#как видно самый высокий показатель lifetime у пользователя 380 он составляет 5969 дней
min_max_fans_lifetime.sort_values('lifetime', ascending=False).head(15)

,userId,min,max,lifetime
140,380,2000-02-01 01:05:25,2016-06-05 19:59:46,5969 days 18:54:21
212,547,2000-11-21 03:25:09,2016-10-16 03:14:04,5807 days 23:48:55
161,427,1999-10-07 21:43:07,2014-11-13 18:04:03,5515 days 20:20:56
3,15,2001-08-16 04:47:19,2016-07-24 03:25:35,5455 days 22:38:16
248,624,2002-04-18 09:57:46,2016-10-16 11:12:53,5295 days 01:15:07
205,529,2000-06-02 16:39:28,2014-08-17 17:30:22,5189 days 00:50:54
254,648,2006-01-28 19:43:56,2016-04-15 21:59:30,3730 days 02:15:34
146,388,1999-12-29 22:59:06,2008-12-11 20:24:01,3269 days 21:24:55
118,311,1998-06-09 14:47:53,2006-05-17 00:13:13,2898 days 09:25:20
179,472,1999-10-04 02:07:18,2007-03-03 21:21:37,2707 days 19:14:19


In [34]:
#среднее значение lifetime для нашей выборки равен 455 дней
min_max_fans_lifetime['lifetime'].mean()

Timedelta('455 days 05:29:08.566539')

In [35]:
#еще намного занимательной статистики
min_max_fans_lifetime.describe()

,userId,lifetime
count,263.000000,263
mean,343.060837,455 days 05:29:08.566539
std,192.531003,1005 days 02:45:56.813082
min,4.000000,0 days 00:03:26
25%,177.500000,0 days 03:08:10.500000
50%,355.000000,27 days 07:11:41
75%,517.000000,397 days 00:33:47
max,671.000000,5969 days 18:54:21


In [36]:
emojize(':thumbs_up:')

'👍'

### Задание 4
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
 
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
 
1. В переменную years запишите список из всех годов с 1950 по 2010.
 
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
- для каждой строки пройдите по всем годам списка years
- если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
- если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
 
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
 
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга